In [22]:
import os
from dotenv import load_dotenv
from huggingface_hub import login
load_dotenv() 
token = os.getenv("HF_API_TOKEN")
login(token=token)
model="microsoft/Phi-3-mini-4k-instruct"


In [ ]:

# Two–Lawyer Agents (Defense & Prosecution)
from __future__ import annotations

from typing import List, Dict
from huggingface_hub import InferenceClient

class LawyerAgent:
    def __init__(self,
                 name: str,
                 system_prompt: str,
                 model: str = model):
        self.name = name
        self.system_prompt = system_prompt.strip()
        self.history: List[Dict[str, str]] = []      # list of {"role": ..., "content": ...}
        self.client = InferenceClient(
            model,
            token=token          # make sure this env‑var is set
        )

    # ---- helper for HF prompt formatting ----------
    def _format_prompt(self, user_msg: str) -> str:
        """
        Formats a full prompt that includes
        * system prompt
        * prior turns
        * new user message
        """
        messages = [{"role": "system", "content": self.system_prompt}]
        messages.extend(self.history)
        messages.append({"role": "user", "content": user_msg})

        # HF text-generation endpoints expect a single string.

        prompt = ""
        for m in messages:
            prompt += f"<|{m['role']}|>\n{m['content']}\n"
        prompt += "<|assistant|>\n"
        return prompt

    # ---- produce a reply --------------------------
    def respond(self, user_msg: str, **gen_kwargs) -> str:
        prompt = self._format_prompt(user_msg)
        completion = self.client.text_generation(
            prompt,
            max_new_tokens=512,
            temperature=0.7,
            do_sample=True,
            stream=False,
            **gen_kwargs
        )
        answer = completion.strip()
        # keep chat memory
        self.history.append({"role": "user", "content": user_msg})
        self.history.append({"role": "assistant", "content": answer})
        return answer



In [24]:
class MainAgent:
    def __init__(self, name: str, system_prompt: str, model: str, token: str):
        self.name = name
        self.system_prompt = system_prompt.strip()
        self.history: List[Dict[str, str]] = []
        self.client = InferenceClient(model, token=token)

    def _format_prompt(self, user_msg: str) -> str:
        messages = [{"role": "system", "content": self.system_prompt}]
        messages.extend(self.history)
        messages.append({"role": "user", "content": user_msg})
        prompt = ""
        for m in messages:
            prompt += f"<|{m['role']}|>\n{m['content']}\n"
        prompt += "<|assistant|>\n"
        return prompt

    def respond(self, user_msg: str, **gen_kwargs) -> str:
        prompt = self._format_prompt(user_msg)
        completion = self.client.text_generation(
            prompt,
            max_new_tokens=512,
            temperature=0.7,
            do_sample=True,
            stream=False,
            **gen_kwargs
        )
        answer = completion.strip()
        self.history.append({"role": "user", "content": user_msg})
        self.history.append({"role": "assistant", "content": answer})
        return answer


In [25]:
# Defendent
class DefendantAgent(MainAgent):
    def __init__(self,model: str,token: str,system_prompt: str): 
        super().__init__("Defendant",system_prompt,model,token)
    def testify(self,question: str) -> str:
        return self.respond(f"As the defendant, answer: {question}")

In [26]:
# Plaintiff
class PlaintiffAgent(MainAgent):
    def __init__(self, model: str,token: str,system_prompt:str):
        super().__init__("Plaintiff",system_prompt,model,token)
    def statement(self) -> str:
        return self.respond("State your grievance and what you want from the court.")

In [27]:
# Judge
class JudgeAgent(MainAgent):
    def __init__(self, model:str,token: str ,system_prompt: str):
        
        super().__init__("Judge",system_prompt,model,token)
    def verdict(self)->str:
        return self.respond("Deliver your verdict and explain your reasoning.")

In [28]:
# Eye witness
class EyewitnessAgent(MainAgent):
    def __init__(self, name: str,witnessed_details:str, model:str,token: str):
        system_prompt = f"""
        You are an eyewitness named {name}.You witnessed the following: {witnessed_details}
        
        Your role is to:
        • Testify truthfully about what you saw, heard, or otherwise perceived
        • Answer questions from both lawyers clearly and accurately
        • Only speak about what you personally witnessed
        • Admit when you are uncertain or don't remember details
        
        Style:
        • Speak in first person ("I saw...")
        • Use descriptive but factual language
        • Maintain consistency in your account
        • Show appropriate emotional reactions based on what you witnessed
        """
        super().__init__(name, system_prompt, model, token)
    
    def testify(self)-> str: 
        return self.respond("Describe in detail what you witnessed regarding this case.")
    
    # def answer_prosecution(self, question: str) -> str:  
    #     return self.respond(f"The prosecution asks: {question}")
    
    # def answer_defense(self, question: str) -> str:
    #     return self.respond(f"The defense attorney asks during cross-examination: {question}")

In [29]:
# All prompts

DEFENSE_SYSTEM = """
You are **Alex Carter**, lead *defense counsel*.
Goals:
• Protect the constitutional rights of the defendant.
• Raise reasonable doubt by pointing out missing evidence or alternative explanations.
• Be respectful to the Court and to opposing counsel.
Style:
• Crisp, persuasive, grounded in precedent and facts provided.
• When citing precedent: give short case name + year (e.g., *Miranda v. Arizona* (1966)).
Ethics:
• Do not fabricate evidence; admit uncertainty when required.
"""

PROSECUTION_SYSTEM = """
You are **Jordan Blake**, *Assistant District Attorney* for the State.
Goals:
• Present the strongest good-faith case against the accused.
• Lay out facts logically, citing exhibits or witness statements when available.
• Anticipate and rebut common defense arguments.
Style:
• Formal but plain English; persuasive, with confident tone.
Ethics:
• Duty is to justice, not merely to win. Concede points when ethically required.
"""

DEFENDANT_PROMPT = """
        You are the defendant. Respond truthfully to all questions about your actions and intentions.
        Only speak when addressed. Admit what you know and clarify misunderstandings.
        """

PLAINTIFF_PROMPT = """
        You are the plaintiff. Clearly explain your grievance, the harm suffered, and what you seek from the court.
        Answer questions honestly and provide details about the incident.
        """

JUDGE_PROMPT = """
        You are the judge. Ensure fairness, rule on objections, explain the law, and make the final decision.
        Remain impartial and base rulings on evidence and law.
        """



In [30]:
def run_trial(case_background: str, eyewitness_details: str = None):
    # initialize agents
    defense = LawyerAgent("Defense", DEFENSE_SYSTEM)
    prosecution = LawyerAgent("Prosecution", PROSECUTION_SYSTEM)
    defendant = DefendantAgent(model=model, token=token, system_prompt=DEFENDANT_PROMPT)
    plaintiff = PlaintiffAgent(model=model, token=token, system_prompt=PLAINTIFF_PROMPT)
    judge = JudgeAgent(model=model, token=token, system_prompt=JUDGE_PROMPT)

    
    # 1. opening Statements
    print("\n=== OPENING STATEMENTS ===")
    print("PROSECUTION:", prosecution.respond(f"Opening statement: {case_background}"))
    print("\nDEFENSE:", defense.respond("Opening statement responding to prosecution"))

    # 2. plaintiff's Case
    print("\n=== PLAINTIFF TESTIMONY ===")
    print("PLAINTIFF:", plaintiff.respond("State your grievance to the court"))

    # 3. eyewitness
    if eyewitness_details:
        eyewitness = EyewitnessAgent("Witness", eyewitness_details, model, token)
        print("\n=== EYEWITNESS TESTIMONY ===")
        print("EYEWITNESS:", eyewitness.testify())

    # 4. defendant's Case
    print("\n=== DEFENDANT TESTIMONY ===")
    print("DEFENDANT:", defendant.testify("Explain your side of the case"))

    # 5. closing Statements
    print("\n=== CLOSING STATEMENTS ===")
    print("PROSECUTION:", prosecution.respond("Final closing argument"))
    print("\nDEFENSE:", defense.respond("Final closing argument"))

    # 6. judge verdict
    print("\n=== JUDGE'S VERDICT ===")
    print("JUDGE:", judge.respond("Deliver your verdict based on the evidence presented"))

In [32]:
case_background = (
    "The State alleges that John Doe stole proprietary algorithms from his former employer "
    "and used them at a competitor. The charge is felony theft of trade secrets. "
    "No physical evidence shows direct copying, but server logs indicate large downloads "
    "two days before Doe resigned."
)
eyewitness_details=(
    "I noticed unusual activity on the server two days before John Doe resigned."
    "His account accessed and downloaded several proprietary algorithm files."
    "He wasn't authorized to work with those files at that time."
    "I reported the incident to my supervisor immediately."
)
run_trial(case_background,eyewitness_details)


=== OPENING STATEMENTS ===
PROSECUTION: "Ladies and Gentlemen of the jury, good morning. Today, we have before us a case of serious concern, where the integrity of our industry's innovation is at stake. The accused, Mr. John Doe, has abused his position of trust and taken what doesn't belong to him—the very foundation of his former employer's success.


This case is not about a simple misplaced item, or an accidental download. It's about a deliberate act of corporate espionage. The evidence is clear when we examine the server logs, which show a pattern of downloads that coincide with the timing of Mr. Doe's resignation. This is not a mere coincidence, but a calculated move, indicative of his intent to benefit a competitor with stolen intellectual property.


We will demonstrate through expert testimony that these algorithms are not generic; they are the result of years of research and development. Their theft undermines our client's investment and that of its stakeholders.


The defen

In [ ]:
import pandas as pd
df = pd.read_csv("/home/sairaj/Desktop/CODING(main folder)/main/Python/Projects/Cynaptics-phase2 Induction task/cases.csv")
df

,Unnamed: 0,id,text
0,0,1989_75,civil appellate jurisdiction civil appeal numb...
1,1,1959_102,civil appellate jurisdiction civil appeal numb...
2,2,1979_76,civil appellate jurisdiction civil appeal numb...
3,3,1991_124,civil appellate jurisdiction civil appeal numb...
4,4,1985_233,original jurisdiction writ petitions number. 8...
...,...,...,...
95,95,1968_146,\nramaswami j. \n\nthese appeals are brought b...
96,96,1992_528,civil appellate jurisdiction special leave pet...
97,97,1958_35,criminal appellate jurisdiction criminal appea...
98,98,1987_407,civil appellate jurisdiction civil appeal numb...
